# Preprocesamiento de Datos
## Proyecto de Clasificación Multiclase

Este notebook contiene el preprocesamiento necesario para preparar los datos:
- División de datos (train/test)
- Escalado de características
- Selección de características
- Manejo de desbalance de clases (si es necesario)

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import pickle

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, mutual_info_classif
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Carga de Datos y Resumen EDA

In [ ]:
# Cargar datos
df = pd.read_csv('../datasets/train.csv')

# Cargar resumen del EDA si existe
try:
    with open('../results/eda_summary.json', 'r') as f:
        eda_summary = json.load(f)
    print("Resumen del EDA:")
    print(json.dumps(eda_summary, indent=2))
except:
    print("No se encontró el resumen del EDA. Ejecuta primero 01_exploratory_data_analysis.ipynb")

print(f"\nDimensiones del dataset: {df.shape}")

In [ ]:
# Separar características y objetivo
feature_cols = [col for col in df.columns if col not in ['id', 'target']]
X = df[feature_cols].copy()
y = df['target'].copy()

print(f"Características: {X.shape[1]}")
print(f"Muestras: {X.shape[0]}")
print(f"\nClases únicas: {y.nunique()}")
print(y.value_counts())

## 2. Codificación de Variables

In [ ]:
# Codificar la variable objetivo si es categórica
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Mapeo de clases:")
for i, class_name in enumerate(label_encoder.classes_):
    print(f"{class_name} -> {i}")

# Guardar el encoder
with open('../models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
    
print("\n✓ Label Encoder guardado")

## 3. División de Datos (Train/Test)

In [ ]:
# División estratificada para mantener la proporción de clases
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

print("Conjunto de entrenamiento:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"\nConjunto de prueba:")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")

# Verificar distribución de clases
print("\nDistribución en train:")
print(pd.Series(y_train).value_counts(normalize=True).sort_index())
print("\nDistribución en test:")
print(pd.Series(y_test).value_counts(normalize=True).sort_index())

## 4. Eliminación de Características con Baja Varianza

In [ ]:
# Eliminar características con varianza casi nula
variance_selector = VarianceThreshold(threshold=0.01)
X_train_var = variance_selector.fit_transform(X_train)
X_test_var = variance_selector.transform(X_test)

# Obtener nombres de características seleccionadas
selected_features_mask = variance_selector.get_support()
selected_features = X_train.columns[selected_features_mask].tolist()

print(f"Características originales: {X_train.shape[1]}")
print(f"Características después de eliminar baja varianza: {X_train_var.shape[1]}")
print(f"Características eliminadas: {X_train.shape[1] - X_train_var.shape[1]}")

# Convertir a DataFrame
X_train_var = pd.DataFrame(X_train_var, columns=selected_features, index=X_train.index)
X_test_var = pd.DataFrame(X_test_var, columns=selected_features, index=X_test.index)

## 5. Escalado de Características

In [ ]:
# StandardScaler (estandarización: media=0, std=1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_var)
X_test_scaled = scaler.transform(X_test_var)

# Convertir a DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=selected_features, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=selected_features, index=X_test.index)

print("✓ Datos escalados usando StandardScaler")
print(f"\nEstadísticas después del escalado (primeras 5 características):")
print(X_train_scaled.iloc[:, :5].describe())

# Guardar el scaler
with open('../models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("\n✓ Scaler guardado")

## 6. Selección de Características (Feature Selection)

In [ ]:
# Seleccionar las K mejores características usando ANOVA F-value
k_best = min(50, X_train_scaled.shape[1])  # Seleccionar hasta 50 características o todas si hay menos

selector_f = SelectKBest(score_func=f_classif, k=k_best)
X_train_selected_f = selector_f.fit_transform(X_train_scaled, y_train)
X_test_selected_f = selector_f.transform(X_test_scaled)

# Obtener características seleccionadas
selected_mask_f = selector_f.get_support()
selected_features_f = [feat for feat, selected in zip(selected_features, selected_mask_f) if selected]

print(f"Características seleccionadas (ANOVA F-value): {len(selected_features_f)}")
print(f"\nTop 10 características por importancia:")
feature_scores = pd.DataFrame({
    'feature': selected_features,
    'score': selector_f.scores_
}).sort_values('score', ascending=False)
print(feature_scores.head(10))

In [ ]:
# Visualizar las mejores características
plt.figure(figsize=(12, 6))
top_features = feature_scores.head(20)
plt.barh(top_features['feature'], top_features['score'], color='steelblue', edgecolor='black')
plt.xlabel('F-Score', fontsize=12)
plt.ylabel('Características', fontsize=12)
plt.title('Top 20 Características por Importancia (ANOVA F-value)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Selección usando Información Mutua
selector_mi = SelectKBest(score_func=mutual_info_classif, k=k_best)
X_train_selected_mi = selector_mi.fit_transform(X_train_scaled, y_train)
X_test_selected_mi = selector_mi.transform(X_test_scaled)

selected_mask_mi = selector_mi.get_support()
selected_features_mi = [feat for feat, selected in zip(selected_features, selected_mask_mi) if selected]

print(f"Características seleccionadas (Mutual Information): {len(selected_features_mi)}")
print(f"\nCaracterísticas comunes entre ambos métodos: {len(set(selected_features_f) & set(selected_features_mi))}")

## 7. Manejo de Desbalance de Clases (Opcional)

In [ ]:
# Verificar si hay desbalance significativo
class_distribution = pd.Series(y_train).value_counts()
balance_ratio = class_distribution.max() / class_distribution.min()

print(f"Ratio de desbalance: {balance_ratio:.2f}")

if balance_ratio > 2:
    print("\n⚠️ Detectado desbalance de clases. Aplicando SMOTE...")
    
    # Aplicar SMOTE
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_selected_f, y_train)
    
    print(f"\nMuestras antes de SMOTE: {X_train_selected_f.shape[0]}")
    print(f"Muestras después de SMOTE: {X_train_resampled.shape[0]}")
    print(f"\nDistribución después de SMOTE:")
    print(pd.Series(y_train_resampled).value_counts().sort_index())
else:
    print("✓ Las clases están relativamente balanceadas. No se aplicará SMOTE.")
    X_train_resampled = X_train_selected_f
    y_train_resampled = y_train

## 8. Guardar Datos Preprocesados

In [ ]:
# Crear directorio si no existe
Path('../data/processed').mkdir(parents=True, exist_ok=True)

# Guardar conjuntos de datos preprocesados
np.save('../data/processed/X_train_scaled.npy', X_train_scaled.values)
np.save('../data/processed/X_test_scaled.npy', X_test_scaled.values)
np.save('../data/processed/X_train_selected.npy', X_train_selected_f)
np.save('../data/processed/X_test_selected.npy', X_test_selected_f)
np.save('../data/processed/X_train_resampled.npy', X_train_resampled)
np.save('../data/processed/y_train.npy', y_train)
np.save('../data/processed/y_test.npy', y_test)
np.save('../data/processed/y_train_resampled.npy', y_train_resampled)

# Guardar nombres de características
with open('../data/processed/selected_features.json', 'w') as f:
    json.dump({
        'all_features': selected_features,
        'selected_features_f': selected_features_f,
        'selected_features_mi': selected_features_mi
    }, f, indent=2)

# Guardar selectores
with open('../models/variance_selector.pkl', 'wb') as f:
    pickle.dump(variance_selector, f)
with open('../models/feature_selector_f.pkl', 'wb') as f:
    pickle.dump(selector_f, f)
with open('../models/feature_selector_mi.pkl', 'wb') as f:
    pickle.dump(selector_mi, f)

print("✓ Datos preprocesados guardados exitosamente en 'data/processed/'")
print("✓ Modelos de preprocesamiento guardados en 'models/'")

## 9. Resumen del Preprocesamiento

In [ ]:
preprocessing_summary = {
    'original_features': len(feature_cols),
    'features_after_variance_threshold': len(selected_features),
    'features_selected_f_classif': len(selected_features_f),
    'features_selected_mutual_info': len(selected_features_mi),
    'train_samples_original': X_train.shape[0],
    'train_samples_resampled': X_train_resampled.shape[0],
    'test_samples': X_test.shape[0],
    'balance_ratio': float(balance_ratio),
    'smote_applied': balance_ratio > 2
}

with open('../results/preprocessing_summary.json', 'w') as f:
    json.dump(preprocessing_summary, f, indent=2)

print("=== RESUMEN DEL PREPROCESAMIENTO ===")
print(json.dumps(preprocessing_summary, indent=2))

## Conclusiones

✓ Datos divididos en conjuntos de entrenamiento y prueba (80/20)

✓ Características escaladas usando StandardScaler

✓ Características con baja varianza eliminadas

✓ Selección de características más relevantes

✓ Manejo de desbalance de clases (si aplica)

✓ Datos listos para el modelado

### Próximo paso:
Construcción y entrenamiento de modelos de clasificación